In [22]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, Model
import matplotlib.pyplot as plt 
import seaborn as sns
import os
import math
import glob
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F  
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm
import re

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import re
from abc import abstractmethod
from einops import repeat
from functools import partial
from inspect import isfunction

In [24]:
# % Used Smartphone: LGE-lge-LG-H815-5.1                                             
# % Smartphone's Accelerometer: LGE Accelerometer - Vendor: BOSCH                    
# % --> Version: 1                                                                   
# % --> Min - Max Delay: 5000us - 65535000us                                         
# % --> Maximum Range: 16.000000263891405 G                                          
# % --> Resolution: 1.2136514986004396E-4 G                                          
                                                                                    
# % SensorTag's Accelerometer: MPU-9250 MEMS MotionTracking Device - Invensense      
# % --> Maximum Range: 16 G                                                          
# % --> Resolution: 0.00024 G                                                        
                                                                                    
# % MAC Address; Sensor_ID; Position; Device Model                                   
# %f8:95:c7:f3:ba:82; 0; RIGHTPOCKET; lge-LG-H815-5.1                                
# %C4:BE:84:70:64:8A; 1; CHEST; SensorTag                                            
# %C4:BE:84:70:0E:80; 3; WRIST; SensorTag                                            
# %C4:BE:84:71:A5:02; 2; WAIST; SensorTag                                            
# %B0:B4:48:B8:77:03; 4; ANKLE; SensorTag                                            
                                                                                    
# % Sensor_Type:                                                                     
# % Accelerometer = 0                                                                
# % Gyroscope = 1                                                                    
# % Magnetometer = 2  

In [25]:
datapath = "../../../CAGE/data/UMAFall_Dataset/"
all_files = glob.glob(os.path.join(datapath, "*.csv"))

In [26]:
def parse_filename(filename):
    pattern = r'UMAFall_Subject_(\d+)_(ADL|Fall)_([a-zA-Z_]+)_(\d+)_(\d{4}-\d{2}-\d{2})_.*\.csv'
    match = re.match(pattern, filename)
    
    if match:
        subject_id = int(match.group(1))
        activity_type = match.group(2)
        activity = match.group(3)
        trial = int(match.group(4))
        date = match.group(5)
        return subject_id, activity_type, activity, trial, date
    return None

In [27]:
def read_all_sensors_file(filepath):
    sensor_data = {
        'RIGHTPOCKET': {'id': None, 'data': []},
        'CHEST': {'id': None, 'data': []},
        'WRIST': {'id': None, 'data': []},
        'WAIST': {'id': None, 'data': []},
        'ANKLE': {'id': None, 'data': []}
    }
    
    with open(filepath, 'r') as f:
        lines = f.readlines()
        for line in lines:
            if ';' in line and any(pos in line for pos in sensor_data.keys()):
                parts = line.strip().split(';')
                if len(parts) >= 3:
                    sensor_id = parts[1].strip()
                    position = parts[2].strip()
                    if position in sensor_data:
                        sensor_data[position]['id'] = sensor_id
        
        data_start = False
        for line in lines:
            if '% TimeStamp; Sample No;' in line:
                data_start = True
                continue
            
            if data_start:
                try:
                    values = [float(v.strip()) for v in line.split(';')]
                    sensor_id = str(int(values[-1]))
                    sensor_type = int(values[-2])
                    data = values[2:5]
                    
                    for position, info in sensor_data.items():
                        if info['id'] == sensor_id:
                            info['data'].append([*data, sensor_type])
                except:
                    continue
    
    for position in sensor_data:
        if sensor_data[position]['data']:
            sensor_data[position]['data'] = np.array(sensor_data[position]['data'])
        else:
            sensor_data[position]['data'] = None
            
    return sensor_data

In [28]:
def process_sensor_data(sensor_data):
    processed_data = {}
    valid_lengths = []
    
    for position, data in sensor_data.items():
        if data['data'] is not None:
            acc_mask = data['data'][:, -1] == 0
            gyro_mask = data['data'][:, -1] == 1
            mag_mask = data['data'][:, -1] == 2
            
            acc_data = data['data'][acc_mask][:, :3] if np.any(acc_mask) else np.array([])
            gyro_data = data['data'][gyro_mask][:, :3] if np.any(gyro_mask) else np.array([])
            mag_data = data['data'][mag_mask][:, :3] if np.any(mag_mask) else np.array([])
            
            if len(acc_data) > 0:
                valid_lengths.append(len(acc_data))
            if len(gyro_data) > 0:
                valid_lengths.append(len(gyro_data))
            if len(mag_data) > 0:
                valid_lengths.append(len(mag_data))
    
    if not valid_lengths:
        return None
        
    min_length = min(valid_lengths)
    
    for position, data in sensor_data.items():
        if data['data'] is not None:
            acc_mask = data['data'][:, -1] == 0
            gyro_mask = data['data'][:, -1] == 1
            mag_mask = data['data'][:, -1] == 2
            
            if np.any(acc_mask):
                acc_data = data['data'][acc_mask][:min_length, :3]
                acc_cols = [f'{position.lower()}_acc_x', f'{position.lower()}_acc_y', f'{position.lower()}_acc_z']
                processed_data.update(dict(zip(acc_cols, acc_data.T)))
            
            if np.any(gyro_mask):
                gyro_data = data['data'][gyro_mask][:min_length, :3]
                gyro_cols = [f'{position.lower()}_gyro_x', f'{position.lower()}_gyro_y', f'{position.lower()}_gyro_z']
                processed_data.update(dict(zip(gyro_cols, gyro_data.T)))
            
            if np.any(mag_mask):
                mag_data = data['data'][mag_mask][:min_length, :3]
                mag_cols = [f'{position.lower()}_mag_x', f'{position.lower()}_mag_y', f'{position.lower()}_mag_z']
                processed_data.update(dict(zip(mag_cols, mag_data.T)))
    
    if not processed_data:
        return None
        
    return pd.DataFrame(processed_data)

In [29]:
from sklearn.model_selection import train_test_split 

def prepare_data(features):
    sensor_columns = [col for col in features.columns if any(x in col for x in ['acc', 'gyro', 'mag'])]
    scaler = StandardScaler()
    features_scaled = features.copy()
    features_scaled[sensor_columns] = scaler.fit_transform(features[sensor_columns])
    features_scaled = features_scaled[sensor_columns + ['activity']]
    
    print("columns : ", features_scaled.columns)
    
    # 1. ADL과 Fall 데이터 분리
    adl_mask = features_scaled['activity'] == 0
    adl_data = features_scaled[adl_mask]
    
    fall_mask = features_scaled['activity'] == 1
    fall_data = features_scaled[fall_mask]
    
    # 2. Window 기반 시계열 데이터 생성
    adl_sequences, adl_labels = split_sequences(adl_data, WINDOW_LENGTH, STRIDE)
    fall_sequences, fall_labels = split_sequences(fall_data, WINDOW_LENGTH, STRIDE)
    
    print(f"ADL sequences shape: {adl_sequences.shape}")
    print(f"Fall sequences shape: {fall_sequences.shape}")
    
    # 3. 훈련/테스트 데이터 분할
    # ADL 데이터: 80%는 훈련용, 20%는 테스트용
    adl_train_data, adl_test_data, adl_train_labels, adl_test_labels = train_test_split(
        adl_sequences, adl_labels, test_size=0.2, random_state=42
    )
    
    # Fall 데이터: 모두 테스트용 (또는 훈련/테스트로 분할 가능)
    # 여기서는 모든 Fall 데이터를 테스트용으로 사용
    # 필요에 따라 아래 주석을 해제하여 Fall 데이터도 훈련/테스트로 분할 가능
    """
    fall_train_data, fall_test_data, fall_train_labels, fall_test_labels = train_test_split(
        fall_sequences, fall_labels, test_size=0.2, random_state=42
    )
    """
    fall_test_data = fall_sequences
    fall_test_labels = fall_labels
    
    # 4. 최종 데이터셋 구성
    # 훈련 데이터: ADL의 훈련 부분만 사용
    train_data = adl_train_data
    train_labels = adl_train_labels
    
    # 테스트 데이터: ADL의 테스트 부분과 Fall 데이터 분리
    test_adl_data = adl_test_data
    test_adl_labels = adl_test_labels
    
    # Fall 데이터를 두 부분으로 나누기 (fall1, fall2)
    # 여기서는 간단히 절반으로 나눕니다
    split_idx = len(fall_test_data) // 2
    test_fall1_data = fall_test_data[:split_idx]
    test_fall1_labels = fall_test_labels[:split_idx]
    test_fall2_data = fall_test_data[split_idx:]
    test_fall2_labels = fall_test_labels[split_idx:]
    
    print(f"Train data shape: {train_data.shape}")
    print(f"Test ADL data shape: {test_adl_data.shape}")
    print(f"Test Fall1 data shape: {test_fall1_data.shape}")
    print(f"Test Fall2 data shape: {test_fall2_data.shape}")
    
    return train_data, test_adl_data, test_fall1_data, test_fall2_data

In [30]:
def split_sequences(sequences, n_steps, stride):
    X, y = [], []
    for i in range(0, len(sequences) - n_steps + 1, stride):
        end_ix = i + n_steps
        
        seq_x = sequences.iloc[i:end_ix].iloc[:, :-1]  
        seq_y = sequences.iloc[end_ix-1].iloc[-1]      
        
        X.append(seq_x.values)
        y.append(seq_y)
    
    return np.array(X), np.array(y)

In [31]:
subject_id = 16
dfs_16 = []

for file in tqdm(all_files):
    filename = os.path.basename(file)
    info = parse_filename(filename)
    
    if info and info[0] == subject_id:
        sensor_data = read_all_sensors_file(file)
        df = process_sensor_data(sensor_data)
        
        if df is not None:
            df['subject'] = info[0]
            df['activity'] = info[2]  
            df['trial'] = info[3]
            dfs_16.append(df)

features_16 = pd.concat(dfs_16, ignore_index=True)
features_16['activity'] = features_16['activity'].map(
    lambda x: 1 if x in ['forwardFall', 'backwardFall', 'lateralFall'] else 0
)
features_16 = features_16.dropna()

100%|█| 746/746 [00:00<00:00, 1246.1


In [32]:
subject_id = 17
dfs_17 = []

for file in tqdm(all_files):
    filename = os.path.basename(file)
    info = parse_filename(filename)
    
    if info and info[0] == subject_id:
        sensor_data = read_all_sensors_file(file)
        df = process_sensor_data(sensor_data)
        
        if df is not None:
            df['subject'] = info[0]
            df['activity'] = info[2]  
            df['trial'] = info[3]
            dfs_17.append(df)

features_17 = pd.concat(dfs_17, ignore_index=True)
features_17['activity'] = features_17['activity'].map(
    lambda x: 1 if x in ['forwardFall', 'backwardFall', 'lateralFall'] else 0
)
features_17 = features_17.dropna()

100%|█| 746/746 [00:01<00:00, 713.00


In [33]:
subject_id = 18
dfs_18 = []

for file in tqdm(all_files):
    filename = os.path.basename(file)
    info = parse_filename(filename)
    
    if info and info[0] == subject_id:
        sensor_data = read_all_sensors_file(file)
        df = process_sensor_data(sensor_data)
        
        if df is not None:
            df['subject'] = info[0]
            df['activity'] = info[2]  
            df['trial'] = info[3]
            dfs_18.append(df)

features_18 = pd.concat(dfs_18, ignore_index=True)
features_18['activity'] = features_18['activity'].map(
    lambda x: 1 if x in ['forwardFall', 'backwardFall', 'lateralFall'] else 0
)
features_18 = features_18.dropna()

100%|█| 746/746 [00:08<00:00, 88.01i


In [34]:
WINDOW_LENGTH = 40
STRIDE = 20

train_data_16, test_adl_data_16, test_fall1_data_16, test_fall2_data_16 = prepare_data(features_16)
train_data_17, test_adl_data_17, test_fall1_data_17, test_fall2_data_17 = prepare_data(features_17)
train_data_18, test_adl_data_18, test_fall1_data_18, test_fall2_data_18 = prepare_data(features_18)

columns :  Index(['rightpocket_acc_x', 'rightpocket_acc_y', 'rightpocket_acc_z',
       'chest_acc_x', 'chest_acc_y', 'chest_acc_z', 'chest_gyro_x',
       'chest_gyro_y', 'chest_gyro_z', 'chest_mag_x', 'chest_mag_y',
       'chest_mag_z', 'wrist_acc_x', 'wrist_acc_y', 'wrist_acc_z',
       'wrist_gyro_x', 'wrist_gyro_y', 'wrist_gyro_z', 'wrist_mag_x',
       'wrist_mag_y', 'wrist_mag_z', 'waist_acc_x', 'waist_acc_y',
       'waist_acc_z', 'waist_gyro_x', 'waist_gyro_y', 'waist_gyro_z',
       'waist_mag_x', 'waist_mag_y', 'waist_mag_z', 'ankle_acc_x',
       'ankle_acc_y', 'ankle_acc_z', 'ankle_gyro_x', 'ankle_gyro_y',
       'ankle_gyro_z', 'ankle_mag_x', 'ankle_mag_y', 'ankle_mag_z',
       'activity'],
      dtype='object')
ADL sequences shape: (73, 40, 39)
Fall sequences shape: (79, 40, 39)
Train data shape: (58, 40, 39)
Test ADL data shape: (15, 40, 39)
Test Fall1 data shape: (39, 40, 39)
Test Fall2 data shape: (40, 40, 39)
columns :  Index(['rightpocket_acc_x', 'rightpocket_acc_

----

In [35]:
EPOCHS = 500
INITIAL_LEARNING_RATE = 1e-4 
DECAY_STEPS = 1000
DECAY_RATE = 0.98
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 32

import math
from functools import partial
from inspect import isfunction

In [36]:
class UMAFallDataset(Dataset):
    def __init__(self, data):
        # Transpose data from [batch, window_length, features] to [batch, features, window_length]
        data = data.transpose(0, 2, 1)
        self.data = torch.FloatTensor(data)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]  # Returns shape [features, window_length]

In [37]:
def exists(x):
    return x is not None


def default(val, d):
    if exists(val):
        return val
    return d() if isfunction(d) else d


def noise_like(shape, device, repeat=False):
    repeat_noise = lambda: torch.randn((1, *shape[1:]), device=device).repeat(
        shape[0], *((1,) * (len(shape) - 1))
    )
    noise = lambda: torch.randn(shape, device=device)
    return repeat_noise() if repeat else noise()


def extract(a, t, x_shape):
    b, *_ = t.shape
    out = a.gather(-1, t)
    return out.reshape(b, *((1,) * (len(x_shape) - 1)))


def make_beta_schedule(
    schedule, n_timestep, linear_start=1e-4, linear_end=2e-2, cosine_s=8e-3
):
    if schedule == "linear":
        betas = (
            torch.linspace(
                linear_start**0.5,
                linear_end**0.5,
                n_timestep,
                dtype=torch.float64,
            )
            ** 2
        )

    elif schedule == "cosine":
        timesteps = (
            torch.arange(n_timestep + 1, dtype=torch.float64) / n_timestep
            + cosine_s
        )
        alphas = timesteps / (1 + cosine_s) * np.pi / 2
        alphas = torch.cos(alphas).pow(2)
        alphas = alphas / alphas[0]
        betas = 1 - alphas[1:] / alphas[:-1]
        betas = np.clip(betas, a_min=0, a_max=0.999)

    elif schedule == "sqrt_linear":
        betas = torch.linspace(
            linear_start, linear_end, n_timestep, dtype=torch.float64
        )
    elif schedule == "sqrt":
        betas = (
            torch.linspace(
                linear_start, linear_end, n_timestep, dtype=torch.float64
            )
            ** 0.5
        )
    else:
        raise ValueError(f"schedule '{schedule}' unknown.")
    return betas.numpy()


def timestep_embedding(timesteps, dim, max_period=10000, repeat_only=False):
    """
    Create sinusoidal timestep embeddings.
    :param timesteps: a 1-D Tensor of N indices, one per batch element.
                    These may be fractional.
    :param dim: the dimension of the output.
    :param max_period: controls the minimum frequency of the embeddings.
    :return: an [N x dim] Tensor of positional embeddings.
    """
    if not repeat_only:
        half = dim // 2
        freqs = torch.exp(
            -math.log(max_period)
            * torch.arange(start=0, end=half, dtype=torch.float32)
            / half
        ).to(device=timesteps.device)
        args = timesteps[:, None].float() * freqs[None]
        embedding = torch.cat([torch.cos(args), torch.sin(args)], dim=-1)
        if dim % 2:
            embedding = torch.cat(
                [embedding, torch.zeros_like(embedding[:, :1])], dim=-1
            )
    else:
        embedding = repeat(timesteps, "b -> b d", d=dim)
    return embedding


def zero_module(module):
    """
    Zero out the parameters of a module and return it.
    """
    for p in module.parameters():
        p.detach().zero_()
    return module

In [38]:
class TimestepBlock(nn.Module):
    @abstractmethod
    def forward(self, x, emb):
        """
        Apply the module to `x` given `emb` timestep embeddings.
        """


class TimestepEmbedSequential(nn.Sequential, TimestepBlock):
    """
    A sequential module that passes timestep embeddings to the children that
    support it as an extra input.
    """

    def forward(self, x, emb, context=None):
        for layer in self:
            if isinstance(layer, TimestepBlock):
                x = layer(x, emb)
            else:
                x = layer(x)
        return x


def Normalize(in_channels):
    return nn.GroupNorm(
        num_groups=32, num_channels=in_channels, eps=1e-6, affine=True
    )


class QKVAttentionLegacy(nn.Module):
    """
    A module which performs QKV attention.
    Matches legacy QKVAttention + input/ouput heads shaping
    """

    def __init__(self, n_heads):
        super().__init__()
        self.n_heads = n_heads

    def forward(self, qkv):
        """
        Apply QKV attention.
        :param qkv: an [N x (H * 3 * C) x T] tensor of Qs, Ks, and Vs.
        :return: an [N x (H * C) x T] tensor after attention.
        """
        bs, width, length = qkv.shape
        assert width % (3 * self.n_heads) == 0
        ch = width // (3 * self.n_heads)
        q, k, v = qkv.reshape(bs * self.n_heads, ch * 3, length).split(
            ch, dim=1
        )
        scale = 1 / math.sqrt(math.sqrt(ch))
        weight = torch.einsum(
            "bct,bcs->bts", q * scale, k * scale
        )  # More stable with f16 than dividing afterwards
        weight = torch.softmax(weight.float(), dim=-1).type(weight.dtype)
        a = torch.einsum("bts,bcs->bct", weight, v)
        return a.reshape(bs, -1, length)


class AttentionBlock(nn.Module):
    """
    An attention block that allows spatial positions to attend to each other.
    """

    def __init__(
        self,
        channels,
        num_heads=1,
        num_head_channels=-1,
        use_checkpoint=False,
    ):
        super().__init__()
        self.channels = channels
        if num_head_channels == -1:
            self.num_heads = num_heads
        else:
            assert channels % num_head_channels == 0, (
                f"q,k,v channels {channels} is "
                f"not divisible by num_head_channels {num_head_channels}"
            )
            self.num_heads = channels // num_head_channels
        self.use_checkpoint = use_checkpoint
        self.norm = Normalize(channels)
        self.qkv = nn.Conv1d(channels, channels * 3, 1)
        self.attention = QKVAttentionLegacy(self.num_heads)

        self.proj_out = zero_module(nn.Conv1d(channels, channels, 1))

    def forward(self, x):
        return self._forward(x)

    def _forward(self, x):
        b, c, *spatial = x.shape
        x = x.reshape(b, c, -1)
        qkv = self.qkv(self.norm(x))
        h = self.attention(qkv)
        h = self.proj_out(h)
        return (x + h).reshape(b, c, *spatial)


class Downsample(nn.Module):
    """
    A downsampling layer with an optional convolution.
    :param channels: channels in the inputs and outputs.
    :param use_conv: a bool determining if a convolution is applied.
    """

    def __init__(self, channels, use_conv, out_channels=None, padding=1):
        super().__init__()
        self.channels = channels
        self.out_channels = out_channels or channels
        self.use_conv = use_conv
        if use_conv:
            self.op = nn.Conv1d(
                self.channels, self.out_channels, 3, stride=2, padding=padding
            )
        else:
            assert self.channels == self.out_channels
            self.op = nn.AvgPool1d(kernel_size=2, stride=2)

    def forward(self, x):
        assert x.shape[1] == self.channels
        return self.op(x)


class Upsample(nn.Module):
    """
    An upsampling layer with an optional convolution.
    :param channels: channels in the inputs and outputs.
    :param use_conv: a bool determining if a convolution is applied.
    """

    def __init__(self, channels, use_conv, out_channels=None, padding=1):
        super().__init__()
        self.channels = channels
        self.out_channels = out_channels or channels
        self.use_conv = use_conv
        if use_conv:
            self.conv = nn.Conv1d(
                self.channels, self.out_channels, 3, padding=padding
            )

    def forward(self, x):
        assert x.shape[1] == self.channels
        x = F.interpolate(x, scale_factor=2, mode="nearest")
        if self.use_conv:
            x = self.conv(x)
        return x


class ResBlock(TimestepBlock):
    """
    A residual block that can optionally change the number of channels.
    """

    def __init__(
        self,
        channels,
        emb_channels,
        dropout,
        out_channels=None,
        use_conv=False,
        use_scale_shift_norm=False,
        up=False,
        down=False,
    ):
        super().__init__()
        self.channels = channels
        self.emb_channels = emb_channels
        self.dropout = dropout
        self.out_channels = out_channels or channels
        self.use_conv = use_conv
        self.use_scale_shift_norm = use_scale_shift_norm

        self.in_layers = nn.Sequential(
            Normalize(channels),
            nn.SiLU(),
            nn.Conv1d(channels, self.out_channels, 3, padding=1),
        )

        self.updown = up or down

        if up:
            self.h_upd = Upsample(channels, False)
            self.x_upd = Upsample(channels, False)
        elif down:
            self.h_upd = Downsample(channels, False)
            self.x_upd = Downsample(channels, False)
        else:
            self.h_upd = self.x_upd = nn.Identity()

        self.emb_layers = nn.Sequential(
            nn.SiLU(),
            nn.Linear(
                emb_channels,
                2 * self.out_channels
                if use_scale_shift_norm
                else self.out_channels,
            ),
        )
        self.out_layers = nn.Sequential(
            Normalize(self.out_channels),
            nn.SiLU(),
            nn.Dropout(p=dropout),
            zero_module(
                nn.Conv1d(self.out_channels, self.out_channels, 3, padding=1)
            ),
        )

        if self.out_channels == channels:
            self.skip_connection = nn.Identity()
        elif use_conv:
            self.skip_connection = nn.Conv1d(
                channels, self.out_channels, 3, padding=1
            )
        else:
            self.skip_connection = nn.Conv1d(channels, self.out_channels, 1)

    def forward(self, x, emb):
        return self._forward(x, emb)

    def _forward(self, x, emb):
        if self.updown:
            in_rest, in_conv = self.in_layers[:-1], self.in_layers[-1]
            h = in_rest(x)
            h = self.h_upd(h)
            x = self.x_upd(x)
            h = in_conv(h)
        else:
            h = self.in_layers(x)
        emb_out = self.emb_layers(emb).type(h.dtype)
        while len(emb_out.shape) < len(h.shape):
            emb_out = emb_out[..., None]
        if self.use_scale_shift_norm:
            out_norm, out_rest = self.out_layers[0], self.out_layers[1:]
            scale, shift = torch.chunk(emb_out, 2, dim=1)
            h = out_norm(h) * (1 + scale) + shift
            h = out_rest(h)
        else:
            h = h + emb_out
            h = self.out_layers(h)
        return self.skip_connection(x) + h

In [39]:
class UNetModel(nn.Module):
    def __init__(
        self,
        image_size=32,
        in_channels=1,
        model_channels=32,
        out_channels=1,
        num_res_blocks=2,
        attention_resolutions=[16, 8],
        dropout=0.1,
        channel_mult=(2, 4, 8),
        num_heads=4,
        use_scale_shift_norm=False,
        resblock_updown=True,
    ):
        super().__init__()

        self.image_size = image_size
        self.in_channels = in_channels
        self.model_channels = model_channels
        self.out_channels = out_channels
        self.num_res_blocks = num_res_blocks
        self.attention_resolutions = attention_resolutions
        self.dropout = dropout
        self.channel_mult = channel_mult
        self.num_heads = num_heads
        self.use_scale_shift_norm = use_scale_shift_norm
        self.resblock_updown = resblock_updown

        time_embed_dim = model_channels * 4
        self.time_embed = nn.Sequential(
            nn.Linear(model_channels, time_embed_dim),
            nn.SiLU(),
            nn.Linear(time_embed_dim, time_embed_dim),
        )

        self.input_blocks = nn.ModuleList([
            TimestepEmbedSequential(nn.Conv1d(in_channels, model_channels, 3, padding=1))
        ])
        input_block_chans = [model_channels]
        ch = model_channels
        ds = 1

        for level, mult in enumerate(channel_mult):
            for _ in range(num_res_blocks):
                layers = [ResBlock(ch, time_embed_dim, dropout, out_channels=mult * model_channels, use_scale_shift_norm=use_scale_shift_norm)]
                ch = mult * model_channels
                if ds in attention_resolutions:
                    layers.append(AttentionBlock(ch, num_heads=num_heads))
                self.input_blocks.append(TimestepEmbedSequential(*layers))
                input_block_chans.append(ch)
            if level != len(channel_mult) - 1:
                out_ch = ch
                self.input_blocks.append(TimestepEmbedSequential(Downsample(ch, True, out_channels=out_ch)))
                ch = out_ch
                input_block_chans.append(ch)
                ds *= 2

        self.middle_block = TimestepEmbedSequential(
            ResBlock(ch, time_embed_dim, dropout, use_scale_shift_norm=use_scale_shift_norm),
            AttentionBlock(ch, num_heads=num_heads),
            ResBlock(ch, time_embed_dim, dropout, use_scale_shift_norm=use_scale_shift_norm),
        )

        self.output_blocks = nn.ModuleList([])
        for level, mult in list(enumerate(channel_mult))[::-1]:
            for i in range(num_res_blocks + 1):
                ich = input_block_chans.pop()
                layers = [ResBlock(ch + ich, time_embed_dim, dropout, out_channels=model_channels * mult, use_scale_shift_norm=use_scale_shift_norm)]
                ch = model_channels * mult
                if ds in attention_resolutions:
                    layers.append(AttentionBlock(ch, num_heads=num_heads))
                if level and i == num_res_blocks:
                    out_ch = ch
                    layers.append(Upsample(ch, True, out_channels=out_ch))
                    ds //= 2
                self.output_blocks.append(TimestepEmbedSequential(*layers))

        self.out = nn.Sequential(
            Normalize(ch),
            nn.SiLU(),
            zero_module(nn.Conv1d(ch, out_channels, 3, padding=1)),
        )

    def forward(self, x, timesteps=None, context=None, y=None):
        assert timesteps is not None, "timesteps must be provided"
        hs = []
        t_emb = timestep_embedding(timesteps, self.model_channels, repeat_only=False)
        emb = self.time_embed(t_emb)

        h = x
        for module in self.input_blocks:
            h = module(h, emb, context)
            hs.append(h)
        h = self.middle_block(h, emb, context)

        for module in self.output_blocks:
            h_pop = hs.pop()
            if h.shape[2] != h_pop.shape[2]:
                h_pop = F.interpolate(h_pop, size=h.shape[2], mode='nearest')
            h = torch.cat([h, h_pop], dim=1)
            h = module(h, emb, context)

        return self.out(h)

In [40]:
class DDPM(nn.Module):
    def __init__(
        self,
        unet_config,
        timesteps: int = 1000,
        beta_schedule="linear",
        loss_type="l2",
        log_every_t=100,
        clip_denoised=False,
        linear_start=1e-4,
        linear_end=2e-2,
        cosine_s=8e-3,
        original_elbo_weight=0.0,
        v_posterior=0.0,
        l_simple_weight=1.0,
        parameterization="eps",
        learn_logvar=False,
        logvar_init=0.0,
    ):
        super().__init__()
        assert parameterization in [
            "eps",
            "x0",
        ], 'currently only supporting "eps" and "x0"'
        self.parameterization = parameterization

        self.model = UNetModel(**unet_config.get("params", dict()))

        self.clip_denoised = clip_denoised
        self.log_every_t = log_every_t

        self.v_posterior = v_posterior
        self.original_elbo_weight = original_elbo_weight
        self.l_simple_weight = l_simple_weight

        self.loss_type = loss_type

        self.register_schedule(
            beta_schedule=beta_schedule,
            timesteps=timesteps,
            linear_start=linear_start,
            linear_end=linear_end,
            cosine_s=cosine_s,
        )

        self.learn_logvar = learn_logvar
        self.logvar = torch.full(
            fill_value=logvar_init, size=(self.num_timesteps,)
        )
        if self.learn_logvar:
            self.logvar = nn.Parameter(self.logvar, requires_grad=True)

    def register_schedule(
        self,
        beta_schedule="linear",
        timesteps=1000,
        linear_start=1e-4,
        linear_end=2e-2,
        cosine_s=8e-3,
    ):
        betas = make_beta_schedule(
            beta_schedule,
            timesteps,
            linear_start=linear_start,
            linear_end=linear_end,
            cosine_s=cosine_s,
        )
        alphas = 1.0 - betas
        alphas_cumprod = np.cumprod(alphas, axis=0)
        alphas_cumprod_prev = np.append(1.0, alphas_cumprod[:-1])

        (timesteps,) = betas.shape
        self.num_timesteps = int(timesteps)
        self.linear_start = linear_start
        self.linear_end = linear_end

        to_torch = partial(torch.tensor, dtype=torch.float32)

        self.register_buffer("betas", to_torch(betas))
        self.register_buffer("alphas_cumprod", to_torch(alphas_cumprod))
        self.register_buffer(
            "alphas_cumprod_prev", to_torch(alphas_cumprod_prev)
        )

        # calculations for diffusion q(x_t | x_{t-1}) and others
        self.register_buffer(
            "sqrt_alphas_cumprod", to_torch(np.sqrt(alphas_cumprod))
        )
        self.register_buffer(
            "sqrt_one_minus_alphas_cumprod",
            to_torch(np.sqrt(1.0 - alphas_cumprod)),
        )
        self.register_buffer(
            "log_one_minus_alphas_cumprod",
            to_torch(np.log(1.0 - alphas_cumprod)),
        )
        self.register_buffer(
            "sqrt_recip_alphas_cumprod",
            to_torch(np.sqrt(1.0 / alphas_cumprod)),
        )
        self.register_buffer(
            "sqrt_recipm1_alphas_cumprod",
            to_torch(np.sqrt(1.0 / alphas_cumprod - 1)),
        )

        # calculations for posterior q(x_{t-1} | x_t, x_0)
        posterior_variance = (1 - self.v_posterior) * betas * (
            1.0 - alphas_cumprod_prev
        ) / (1.0 - alphas_cumprod) + self.v_posterior * betas
        self.register_buffer(
            "posterior_variance", to_torch(posterior_variance)
        )
        self.register_buffer(
            "posterior_log_variance_clipped",
            to_torch(np.log(np.maximum(posterior_variance, 1e-20))),
        )
        self.register_buffer(
            "posterior_mean_coef1",
            to_torch(
                betas * np.sqrt(alphas_cumprod_prev) / (1.0 - alphas_cumprod)
            ),
        )
        self.register_buffer(
            "posterior_mean_coef2",
            to_torch(
                (1.0 - alphas_cumprod_prev)
                * np.sqrt(alphas)
                / (1.0 - alphas_cumprod)
            ),
        )

        if self.parameterization == "eps":
            lvlb_weights = self.betas**2 / (
                2
                * self.posterior_variance
                * to_torch(alphas)
                * (1 - self.alphas_cumprod)
            )
        elif self.parameterization == "x0":
            lvlb_weights = (
                0.5
                * np.sqrt(torch.Tensor(alphas_cumprod))
                / (2.0 * 1 - torch.Tensor(alphas_cumprod))
            )
        else:
            raise NotImplementedError("mu not supported")
        lvlb_weights[0] = lvlb_weights[1]
        self.register_buffer("lvlb_weights", lvlb_weights, persistent=False)
        assert not torch.isnan(self.lvlb_weights).all()

    def q_mean_variance(self, x_start, t):
        """
        Get the distribution q(x_t | x_0).
        :param x_start: the [N x C x ...] tensor of
        noiseless inputs.
        :param t: the number of diffusion steps (minus 1).
        Here, 0 means one step.
        :return: A tuple (mean, variance, log_variance),
        all of x_start's shape.
        """
        mean = extract(self.sqrt_alphas_cumprod, t, x_start.shape) * x_start
        variance = extract(1.0 - self.alphas_cumprod, t, x_start.shape)
        log_variance = extract(
            self.log_one_minus_alphas_cumprod, t, x_start.shape
        )
        return mean, variance, log_variance

    def predict_start_from_noise(self, x_t, t, noise):
        return (
            extract(self.sqrt_recip_alphas_cumprod, t, x_t.shape) * x_t
            - extract(self.sqrt_recipm1_alphas_cumprod, t, x_t.shape) * noise
        )

    def q_posterior(self, x_start, x_t, t):
        """
        Compute the mean and variance of the diffusion posterior:
            q(x_{t-1} | x_t, x_0)
        """
        posterior_mean = (
            extract(self.posterior_mean_coef1, t, x_t.shape) * x_start
            + extract(self.posterior_mean_coef2, t, x_t.shape) * x_t
        )
        posterior_variance = extract(self.posterior_variance, t, x_t.shape)
        posterior_log_variance_clipped = extract(
            self.posterior_log_variance_clipped, t, x_t.shape
        )
        return (
            posterior_mean,
            posterior_variance,
            posterior_log_variance_clipped,
        )

    def p_mean_variance(self, x, t, clip_denoised: bool, return_x0=False):
        """
        Apply the model to get p(x_{t-1} | x_t)
        :param model: the model, which takes a signal and a batch of timesteps
                    as input.
        :param x: the [N x C x ...] tensor at time t.
        :param t: a 1-D Tensor of timesteps.
        :param clip_denoised: if True, clip the denoised signal into [-1, 1].
        """
        model_out = self.model(x, t)
        if self.parameterization == "eps":
            x_recon = self.predict_start_from_noise(x, t=t, noise=model_out)
        elif self.parameterization == "x0":
            x_recon = model_out

        if clip_denoised:
            x_recon.clamp_(-1.0, 1.0)

        (
            model_mean,
            posterior_variance,
            posterior_log_variance,
        ) = self.q_posterior(x_start=x_recon, x_t=x, t=t)
        if return_x0:
            return (
                model_mean,
                posterior_variance,
                posterior_log_variance,
                x_recon,
            )
        else:
            return model_mean, posterior_variance, posterior_log_variance

    @torch.no_grad()
    def p_sample(
        self,
        x,
        t,
        clip_denoised=True,
        repeat_noise=False,
        return_x0=False,
        temperature=1.0,
        noise_dropout=0.0,
    ):
        """
        Sample x_{t-1} from the model at the given timestep.
        :param x: the current tensor at x_{t-1}.
        :param t: the value of t, starting at 0 for the first diffusion step.
        :param clip_denoised: if True, clip the x_start prediction to [-1, 1].
        """

        b, *_, device = *x.shape, x.device
        outputs = self.p_mean_variance(
            x=x,
            t=t,
            clip_denoised=clip_denoised,
            return_x0=return_x0,
        )
        if return_x0:
            model_mean, _, model_log_variance, x0 = outputs
        else:
            model_mean, _, model_log_variance = outputs

        noise = noise_like(x.shape, device, repeat_noise) * temperature
        if noise_dropout > 0.0:
            noise = torch.nn.functional.dropout(noise, p=noise_dropout)
        # no noise when t == 0
        nonzero_mask = (1 - (t == 0).float()).reshape(
            b, *((1,) * (len(x.shape) - 1))
        )
        if return_x0:
            return (
                model_mean
                + nonzero_mask * (0.5 * model_log_variance).exp() * noise,
                x0,
            )
        else:
            return (
                model_mean
                + nonzero_mask * (0.5 * model_log_variance).exp() * noise
            )

    @torch.no_grad()
    def p_sample_loop(self, shape, return_intermediates=False):
        device = self.betas.device

        b = shape[0]
        img = torch.randn(shape, device=device)
        intermediates = [img]

        for i in tqdm(
            reversed(range(0, self.num_timesteps)),
            desc="sampling loop time step",
            total=self.num_timesteps,
        ):
            img = self.p_sample(
                img,
                torch.full((b,), i, device=device, dtype=torch.long),
                clip_denoised=self.clip_denoised,
            )
            if i % self.log_every_t == 0 or i == self.num_timesteps - 1:
                intermediates.append(img)
        if return_intermediates:
            return img, intermediates
        return img

    @torch.no_grad()
    def sample(self, batch_size=16, return_intermediates=False):
        image_size = self.image_size
        channels = self.channels
        return self.p_sample_loop(
            (batch_size, channels, image_size, image_size),
            return_intermediates=return_intermediates,
        )

    def q_sample(self, x_start, t, noise=None):
        """
        Diffuse the data for a given number of diffusion steps.
        In other words, sample from q(x_t | x_0).
        :param x_start: the initial data batch.
        :param t: the number of diffusion steps (minus 1). Here,
        0 means one step.
        :param noise: if specified, the split-out normal noise.
        :return: A noisy version of x_start.
        """
        noise = default(noise, lambda: torch.randn_like(x_start))

        return (
            extract(self.sqrt_alphas_cumprod, t, x_start.shape) * x_start
            + extract(self.sqrt_one_minus_alphas_cumprod, t, x_start.shape)
            * noise
        )

    def get_loss(self, pred, target, mean=True):
        if self.loss_type == "l1":
            loss = (target - pred).abs()
            if mean:
                loss = loss.mean()
        elif self.loss_type == "l2":
            if mean:
                loss = torch.nn.functional.mse_loss(target, pred)
            else:
                loss = torch.nn.functional.mse_loss(
                    target, pred, reduction="none"
                )
        else:
            raise NotImplementedError("unknown loss type '{loss_type}'")

        return loss

    def p_losses(self, x_start, t, noise=None):
        noise = default(noise, lambda: torch.randn_like(x_start))
        x_noisy = self.q_sample(x_start=x_start, t=t, noise=noise)
        model_output = self.model(x_noisy, t)

        loss_dict = {}
        if self.parameterization == "eps":
            target = noise
        elif self.parameterization == "x0":
            target = x_start
        else:
            raise NotImplementedError(
                f"Paramterization {self.parameterization} not yet supported"
            )

        loss_simple = self.get_loss(model_output, target, mean=False).mean(
            dim=[1, 2]
        )
        loss_dict.update({f"loss_simple": loss_simple.mean()})

        logvar_t = self.logvar[t].to(x_start.device)
        loss = loss_simple / torch.exp(logvar_t) + logvar_t
        if self.learn_logvar:
            loss_dict.update({f"loss_gamma": loss.mean()})
            loss_dict.update({"logvar": self.logvar.data.mean()})

        loss = self.l_simple_weight * loss.mean()

        loss_vlb = self.get_loss(model_output, target, mean=False).mean(
            dim=(1, 2)
        )
        loss_vlb = (self.lvlb_weights[t] * loss_vlb).mean()
        loss_dict.update({f"loss_vlb": loss_vlb})
        loss += self.original_elbo_weight * loss_vlb
        loss_dict.update({f"loss": loss})

        return loss, loss_dict

    def forward(self, x, *args, **kwargs):
        t = torch.randint(
            0, self.num_timesteps, (x.shape[0],), device=x.device
        ).long()
        return self.p_losses(x, t, *args, **kwargs)

    def configure_optimizers(self):
        self.learning_rate = 1e-4
        lr = self.learning_rate
        params = list(self.model.parameters())
        if self.learn_logvar:
            print("Diffusion model optimizing logvar")
            params.append(self.logvar)
        opt = torch.optim.AdamW(params, lr=lr)
        return opt

In [41]:
def train_ddpm_ppg(model, train_loader, optimizer, num_epochs, device):
    for epoch in range(num_epochs):
        epoch_loss = 0
        n_batches = len(train_loader)
        
        pbar = tqdm(enumerate(train_loader), 
                   desc=f'Epoch {epoch+1}/{num_epochs}', 
                   total=n_batches,
                   leave=False)
        
        for i, batch in pbar:
            optimizer.zero_grad()
            x = batch.to(device)
            loss, loss_dict = model(x)
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
            pbar.set_postfix({"Loss": f"{loss.item():.4f}"})
        
        avg_loss = epoch_loss / n_batches
        print(f'Epoch {epoch+1}/{num_epochs} - Avg Loss: {avg_loss:.4f}')

In [42]:
unet_config = {
    "params": {
        "image_size": 40, 
        "in_channels": 39, 
        "model_channels": 32,
        "out_channels": 39,
        "num_res_blocks": 2,
        "attention_resolutions": [10, 5],
        "dropout": 0.1,
        "channel_mult": (2, 4, 8),
        "num_heads": 4,
        "use_scale_shift_norm": True
    }
}

ddpm1 = DDPM(unet_config=unet_config, 
            timesteps=1000,
            beta_schedule="linear").to(device)

optimizer1 = ddpm1.configure_optimizers()

print("\nSubject 16 Training:")
train_dataset_16 = UMAFallDataset(train_data_16)
train_loader_16 = DataLoader(train_dataset_16, batch_size=BATCH_SIZE, shuffle=True)

train_ddpm_ppg(ddpm1, train_loader_16, optimizer1, EPOCHS, device)


Subject 16 Training:


Epoch 1/500 - Avg Loss: 0.9944


Epoch 2/500 - Avg Loss: 0.9963


Epoch 3/500 - Avg Loss: 0.9935


Epoch 4/500 - Avg Loss: 0.9940


...

Epoch 496/500 - Avg Loss: 0.4290


Epoch 497/500 - Avg Loss: 0.4424


Epoch 498/500 - Avg Loss: 0.4542


Epoch 499/500 - Avg Loss: 0.4300


Epoch 500/500 - Avg Loss: 0.4312


In [43]:
ddpm2 = DDPM(unet_config=unet_config, 
            timesteps=1000,
            beta_schedule="linear").to(device)

optimizer2 = ddpm2.configure_optimizers()

print("\nSubject 17 Training:")
train_dataset_17 = UMAFallDataset(train_data_17)
train_loader_17 = DataLoader(train_dataset_17, batch_size=BATCH_SIZE, shuffle=True)

train_ddpm_ppg(ddpm2, train_loader_17, optimizer2, EPOCHS, device)


Subject 17 Training:


Epoch 1/500 - Avg Loss: 0.9922


Epoch 2/500 - Avg Loss: 0.9932


Epoch 3/500 - Avg Loss: 0.9919


Epoch 4/500 - Avg Loss: 0.9894


...

Epoch 496/500 - Avg Loss: 0.3988


Epoch 497/500 - Avg Loss: 0.3808


Epoch 498/500 - Avg Loss: 0.3773


Epoch 499/500 - Avg Loss: 0.3830


Epoch 500/500 - Avg Loss: 0.4093


In [45]:
ddpm3 = DDPM(unet_config=unet_config, 
            timesteps=1000,
            beta_schedule="linear").to(device)

optimizer3 = ddpm3.configure_optimizers()

print("\nSubject 18 Training:")
train_dataset_18 = UMAFallDataset(train_data_18)
train_loader_18 = DataLoader(train_dataset_18, batch_size=BATCH_SIZE, shuffle=True)

train_ddpm_ppg(ddpm3, train_loader_18, optimizer3, EPOCHS-400, device)



Subject 18 Training:


Epoch 1/100 - Avg Loss: 0.9874


Epoch 2/100 - Avg Loss: 0.9641


Epoch 3/100 - Avg Loss: 0.9354


Epoch 4/100 - Avg Loss: 0.9133


...

Epoch 96/100 - Avg Loss: 0.3753


Epoch 97/100 - Avg Loss: 0.3865


Epoch 98/100 - Avg Loss: 0.4011


Epoch 99/100 - Avg Loss: 0.3921


Epoch 100/100 - Avg Loss: 0.3791


---

In [ ]:
def detect_anomalies(model, data_loader, device, threshold=None):
    model.eval()
    anomaly_scores = []
    
    with torch.no_grad():
        for batch in data_loader:
            x_0 = batch.to(device)
            batch_size = x_0.shape[0]
            
            t = torch.randint(
                0, model.num_timesteps, (batch_size,), device=device
            ).long()
            
            noise = torch.randn_like(x_0)
            x_t = model.q_sample(x_0, t, noise)
            
            predicted_noise = model.model(x_t, t)
            
            if model.parameterization == "eps":
                loss = F.mse_loss(predicted_noise, noise, reduction='none')
            else:  
                loss = F.mse_loss(predicted_noise, x_0, reduction='none')
                
            loss = loss.mean(dim=(1, 2))
            anomaly_scores.extend(loss.cpu().numpy())
    
    if threshold is None:
        threshold = np.percentile(anomaly_scores, 95)
    
    anomalies = [score > threshold for score in anomaly_scores]
    
    return anomalies, anomaly_scores, threshold

In [ ]:
def evaluate_model(model, test_adl_data, test_fall_data):
    normal_dataset = UMAFallDataset(test_adl_data)
    normal_loader = DataLoader(normal_dataset, batch_size=BATCH_SIZE)
    
    abnormal_dataset = UMAFallDataset(np.concatenate([test_fall_data[0], test_fall_data[1]]))
    abnormal_loader = DataLoader(abnormal_dataset, batch_size=BATCH_SIZE)
    
    _, normal_scores, _ = detect_anomalies(model, normal_loader, device)
    _, abnormal_scores, _ = detect_anomalies(model, abnormal_loader, device)
    
    return normal_scores, abnormal_scores

In [ ]:
normal_scores_16, abnormal_scores_16 = evaluate_model(ddpm1, test_adl_data_16, 
                                                     (test_fall1_data_16, test_fall2_data_16))
normal_scores_17, abnormal_scores_17 = evaluate_model(ddpm2, test_adl_data_17, 
                                                     (test_fall1_data_17, test_fall2_data_17))
normal_scores_18, abnormal_scores_18 = evaluate_model(ddpm3, test_adl_data_18, 
                                                     (test_fall1_data_18, test_fall2_data_18))

In [ ]:
plt.figure(figsize=(20, 5))

# Subject 16
plt.subplot(1, 3, 1)
plt.hist(normal_scores_16, bins=50, alpha=0.5, color='blue', label='Normal')
plt.hist(abnormal_scores_16, bins=50, alpha=0.5, color='red', label='Abnormal')
threshold_16 = np.percentile(normal_scores_16, 95)
plt.axvline(threshold_16, color='black', linestyle='--', label='Threshold')
plt.title('Subject 16 - MSE Distribution')
plt.xlabel('Anomaly Score (MSE)')
plt.ylabel('Count')
plt.legend()

# Subject 17
plt.subplot(1, 3, 2)
plt.hist(normal_scores_17, bins=50, alpha=0.5, color='blue', label='Normal')
plt.hist(abnormal_scores_17, bins=50, alpha=0.5, color='red', label='Abnormal')
threshold_17 = np.percentile(normal_scores_18, 95)
plt.axvline(threshold_17, color='black', linestyle='--', label='Threshold')
plt.title('Subject 17 - MSE Distribution')
plt.xlabel('Anomaly Score (MSE)')
plt.ylabel('Count')
plt.legend()

# Subject 18
plt.subplot(1, 3, 3)
plt.hist(normal_scores_18, bins=50, alpha=0.5, color='blue', label='Normal')
plt.hist(abnormal_scores_18, bins=50, alpha=0.5, color='red', label='Abnormal')
threshold_18 = np.percentile(normal_scores_18, 95)
plt.axvline(threshold_18, color='black', linestyle='--', label='Threshold')
plt.title('Subject 18 - MSE Distribution')
plt.xlabel('Anomaly Score (MSE)')
plt.ylabel('Count')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
def create_error_df(normal_scores, abnormal_scores):
    return pd.DataFrame({
        'reconstruction_error': np.concatenate([normal_scores, abnormal_scores]),
        'true_class': np.concatenate([np.zeros(len(normal_scores)), np.ones(len(abnormal_scores))])
    })

error_df_16 = create_error_df(normal_scores_16, abnormal_scores_16)
error_df_17 = create_error_df(normal_scores_17, abnormal_scores_17)
error_df_18 = create_error_df(normal_scores_18, abnormal_scores_18)

In [ ]:
plt.figure(figsize=(20, 5))

plt.subplot(1, 3, 1)
sns.boxplot(x='true_class', y='reconstruction_error', data=error_df_16, showfliers=False)
plt.title('Subject 16 - Error Distribution')
plt.ylabel('Reconstruction Error')
plt.xlabel('Class (0: Normal, 1: Abnormal)')

plt.subplot(1, 3, 2)
sns.boxplot(x='true_class', y='reconstruction_error', data=error_df_17, showfliers=False)
plt.title('Subject 17 - Error Distribution')
plt.ylabel('Reconstruction Error')
plt.xlabel('Class (0: Normal, 1: Abnormal)')

plt.subplot(1, 3, 3)
sns.boxplot(x='true_class', y='reconstruction_error', data=error_df_18, showfliers=False)
plt.title('Subject 18 - Error Distribution')
plt.ylabel('Reconstruction Error')
plt.xlabel('Class (0: Normal, 1: Abnormal)')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20, 5))

# Subject 16
plt.subplot(1, 3, 1)
groups = error_df_16.groupby('true_class')
for name, group in groups:
    plt.plot(group.index, group.reconstruction_error, marker='^', ms=3.5, linestyle='',
             label="Normal" if name == 0 else "Fall")
plt.hlines(threshold_16, plt.xlim()[0], plt.xlim()[1], colors="r", zorder=100, label='Threshold')
plt.title("Subject 16 - Reconstruction Error by Class")
plt.ylabel("Reconstruction Error")
plt.xlabel("Data Point Index")
plt.legend()

# Subject 17
plt.subplot(1, 3, 2)
groups = error_df_17.groupby('true_class')
for name, group in groups:
    plt.plot(group.index, group.reconstruction_error, marker='^', ms=3.5, linestyle='',
             label="Normal" if name == 0 else "Fall")
plt.hlines(threshold_17, plt.xlim()[0], plt.xlim()[1], colors="r", zorder=100, label='Threshold')
plt.title("Subject 17 - Reconstruction Error by Class")
plt.ylabel("Reconstruction Error")
plt.xlabel("Data Point Index")
plt.legend()

# Subject 18
plt.subplot(1, 3, 3)
groups = error_df_18.groupby('true_class')
for name, group in groups:
    plt.plot(group.index, group.reconstruction_error, marker='^', ms=3.5, linestyle='',
             label="Normal" if name == 0 else "Fall")
plt.hlines(threshold_18, plt.xlim()[0], plt.xlim()[1], colors="r", zorder=100, label='Threshold')
plt.title("Subject 18 - Reconstruction Error by Class")
plt.ylabel("Reconstruction Error")
plt.xlabel("Data Point Index")
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(20, 5))

# Subject 16
plt.subplot(1, 3, 1)
y_pred_16 = [1 if e > threshold_16 else 0 for e in error_df_16.reconstruction_error.values]
conf_matrix_16 = confusion_matrix(error_df_16.true_class, y_pred_16)
sns.heatmap(conf_matrix_16, xticklabels=["Normal", "Abnormal"], 
            yticklabels=["Normal", "Abnormal"], annot=True, fmt="d")
plt.title("Subject 16 - Confusion Matrix")
plt.ylabel('True Class')
plt.xlabel('Predicted Class')

# Subject 17
plt.subplot(1, 3, 2)
y_pred_17 = [1 if e > threshold_17 else 0 for e in error_df_17.reconstruction_error.values]
conf_matrix_17 = confusion_matrix(error_df_17.true_class, y_pred_17)
sns.heatmap(conf_matrix_17, xticklabels=["Normal", "Abnormal"], 
            yticklabels=["Normal", "Abnormal"], annot=True, fmt="d")
plt.title("Subject 17 - Confusion Matrix")
plt.ylabel('True Class')
plt.xlabel('Predicted Class')

# Subject 18
plt.subplot(1, 3, 3)
y_pred_18 = [1 if e > threshold_18 else 0 for e in error_df_18.reconstruction_error.values]
conf_matrix_18 = confusion_matrix(error_df_18.true_class, y_pred_18)
sns.heatmap(conf_matrix_18, xticklabels=["Normal", "Abnormal"], 
            yticklabels=["Normal", "Abnormal"], annot=True, fmt="d")
plt.title("Subject 18 - Confusion Matrix")
plt.ylabel('True Class')
plt.xlabel('Predicted Class')

plt.tight_layout()
plt.show()

In [ ]:
for subject_id, error_df, threshold in [
    (16, error_df_16, threshold_16),
    (17, error_df_17, threshold_17),
    (18, error_df_18, threshold_18)
]:
    y_pred = [1 if e > threshold else 0 for e in error_df.reconstruction_error.values]
    y_true = error_df.true_class.values
    
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    print(f'\nSubject {subject_id} Performance:')
    print(f'Accuracy Score : {accuracy:.4f}')
    print(f'Precision     : {precision:.4f}')
    print(f'Recall        : {recall:.4f}')
    print(f'F1 Score      : {f1:.4f}')